In [ ]:
!pip install nltk

: 

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle


In [ ]:
#import tensorflow as tf
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
#from keras.optimizers import legacy
from keras.optimizers import SGD
#from tensorflow.keras import Input
#from tensorflow.keras.utils import to_categorical
import random

In [ ]:
#initialize lists
words =[]
classes = []
documents = []
ignore_words=['?','!','@','$']


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#use json
data_file = open('D:\NutriAI\src\Components\Chatbot\intents (1).json', encoding='utf-8').read()
intents = json.loads(data_file)

In [ ]:
#populating the lists
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #take rach word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)

        #adding documents
        documents.append((w,intent['tag']))


        #adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print(len(documents), "Documents: ")
#print(" /n")

print(len(classes), "Classes:", classes)
#print(" /n")

print(len(words), "Unique Lemmatized words", words)
#print(" /n")

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes, open('classes.pkl','wb'))

916 Documents: 
255 Classes: ['     Support Vector Machine', '   Decision Trees', '  Perceptron', '  Regularization in Machine Learning', '  applications of supervised learning', '  basic data structures', " 'action' in reinforcement learning", " 'agent' in reinforcement learning", " 'value' in reinforcement learning", ' Classification model', ' Gradient Descent', ' Inductive Logic Programming', ' Polynomial Regression', ' Regularization', ' Ridge Regression technique', ' Types of machine learning', ' advantages of using Polynomial Regression over Linear Regression', ' applications of self-supervised learning', ' components of Bayesian logic program', ' difference between heuristic for rule learning and heuristics', ' disadvantages of supervised learning', ' imbalanced datasheet', ' issues to consider in supervised learning', ' standard supervised learning', 'A/B Testing', 'Algorithm techniques in Machine Learning', 'Appreciation', 'Apriori algorithm', 'Areas of Problems', 'Concern', '

In [ ]:
#initialise training data
training = []
output_empty = [0] * len(classes)

for doc in documents:
    #initializing the bag of words
    bag = []
    #list of tokenized words for pattern
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]


    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)


    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag,output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
# Convert the list of bags-of-words and output labels to numpy arrays
train_x = np.array([np.array(x[0]) for x in training], dtype=np.float32)  # Ensure uniform input size
train_y = np.array([np.array(x[1]) for x in training], dtype=np.float32)  # Ensure uniform output size

print("Training data created")

# Check if all arrays have the correct shape
print("train_x shape:", train_x.shape)
print("train_y shape:", train_y.shape)

Training data created
train_x shape: (916, 688)
train_y shape: (916, 255)


In [ ]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(learning_rate= 0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


In [ ]:
# Check the shape of train_y before conversion (optional)
#print("train_y shape before conversion:", train_y.shape)

# Assuming train_y contains a list of integer class labels
#num_classes = len(set(train_y))

# One-hot encode the labels (if necessary)
#if train_y.ndim == 1:  # Check if train_y is a 1D array
#   train_y_encoded = to_categorical(train_y, num_classes)
#else:
#    train_y_encoded = train_y  # No need to encode if already 2D



hist = model.fit(train_x, train_y, epochs=200, batch_size=5, verbose=1)

model.save('chatbot_model.h5', hist)

print('model created')

Epoch 1/200
184/184 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.0041 - loss: 5.5457
Epoch 2/200
184/184 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0079 - loss: 5.5105
Epoch 3/200
184/184 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0407 - loss: 5.4589
Epoch 4/200
184/184 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0148 - loss: 5.3578
Epoch 5/200
184/184 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0254 - loss: 5.2465
Epoch 6/200
184/184 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0574 - loss: 4.9999
Epoch 7/200
184/184 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0682 - loss: 4.8140
Epoch 8/200
184/184 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1047 - loss: 4.4069
Epoch 9/200
184/184 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1285 - loss: 4.2066
Epoch 10/200
184/184 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1942 - loss: 3.7182
Epoch 11/200
184/184 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1778 - loss: 3.6458
Epoch 12/200
184/184 ━━━━━━━━━━━━━━━━━━━━

model created
